# Final Project: Disaster Tweet Classification

In [ ]:
__author__ = "Kevin Guo, Pranav Sriram, Raymond Yao"
__version__ = "CS224u, Stanford, Spring 2021"

## Data Pre-Processing

In [ ]:
import numpy as np
import pandas as pd
import re
from transformers import BertModel, BertTokenizer
from transformers import AutoTokenizer
import utils
import torch
import torch.nn as nn
from torch_shallow_neural_classifier import TorchShallowNeuralClassifier
from datasets import Dataset
from datasets import load_dataset
#from datasets import train_test_split
from transformers import TrainingArguments
from transformers import Trainer
from torch.utils.data import DataLoader
from transformers import AdamW
from transformers import get_scheduler
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split

In [ ]:
# Set random seeds
utils.fix_random_seeds()

In [ ]:
weights_name = 'bert-base-cased'

In [ ]:
bert_tokenizer = BertTokenizer.from_pretrained(weights_name)

In [ ]:
bert_model = BertModel.from_pretrained(weights_name)

In [ ]:
def bert_phi(text):
    input_ids = bert_tokenizer.encode(text, add_special_tokens=True)
    X = torch.tensor([input_ids])
    with torch.no_grad():
        reps = bert_model(X)
        return reps.last_hidden_state.squeeze(0).numpy()

In [ ]:
def bert_classifier_phi(text):
    reps = bert_phi(text)
    #return reps.mean(axis=0)  # Another good, easy option.
    return reps[0]

In [ ]:
#all_data = pd.read_csv('real_and_fake_news.csv')
all_data['text'] = all_data['text'].apply(lambda x: re.sub(r'https?\S+', '', x))
relevant_data, unneeded = np.split(all_data.sample(frac=1, random_state=42), [10000])
train, dev, test = np.split(relevant_data, [int(.8*len(relevant_data)), int(.9*len(relevant_data))])

In [ ]:
print(train)
print(dev)
print(test)

In [ ]:
X_str_train = train.text.values
print(len(X_str_train))
y_train = train.label.values

X_str_dev = dev.text.values
print(len(X_str_dev))
y_dev = dev.label.values

In [ ]:
%time X_train = [bert_classifier_phi(text) for text in X_str_train]

In [ ]:
%time X_dev = [bert_classifier_phi(text) for text in X_str_dev]

In [ ]:
model = TorchShallowNeuralClassifier(
    early_stopping=True,
    hidden_dim=300)

In [ ]:
%time _ = model.fit(X_train, y_train)

In [ ]:
preds = model.predict(X_dev)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_dev, preds, digits=3))